## All of the functions for the solver

In [1]:
import input_functions as inp
import tsp_routines
from clustering.funcs import k_cluster
from clustering.funcs import best_dropoff
from input_functions.funcs import create_new_graph

import numpy as np
import pandas as pd
import networkx as nx
import pickle
import os
import sys

import matplotlib.pyplot as plt

from joblib import Parallel, delayed
import multiprocessing

cpus = multiprocessing.cpu_count()

In [41]:
def map_dict_entries(input_dict,nodemapper):
    """
    maps the keys and values of the input dict using the nodemapper dict.
    """
    output_dict = dict()
    for key in input_dict.keys():
        mapped_key = nodemapper[key]
        output_dict.update({mapped_key:[]})
        for vertex in input_dict[key]:
            output_dict[mapped_key].append(nodemapper[vertex])
    return output_dict

def tsp_solution_to_path(G,tsp_route):
    """
    converts the given tsp sequence to be followed by the car into a 
    path in the graph G
    Input:
    G - undirected weighted input graph
    tsp_route - list of vertices specifying the route to be followed by the car
    """
    prev = tsp_route[0]
    final_path = []
    final_path.append(prev)
    for vertex in tsp_route[1:]:
        path = nx.shortest_path(G,prev,vertex,weight='weight')
        final_path += path[1:]
        prev = vertex
    return final_path

def add_vertex_to_clusters(clusters,vertex):
    """
    add the given vertex to each cluster.
    Input:
    clusters - dict where the keys are vertices which are cluster centers and the values are a list of 
                vertices belonging to this cluster
    vertex - the vertex to be added to each list in `clusters`
    """
    for key in clusters:
        clusters[key].append(vertex)
        
def get_dropoff_vertices(G, clusters):
    best_dropoffs = []
    for key in clusters:
        dropoff = best_dropoff(G,clusters[key])
        best_dropoffs.append(dropoff)
    return best_dropoffs
        
def solve_by_clustering(graph,homes,source,num_clusters):
    """
    return the route to be followed by the car as it drops off TAs.
    Inputs:
    graph - input graph
    homes - list of vertices in `graph` that are marked as homes
    source - vertex in `graph` that is the start and end of the path followed by the car
    num_clusters - the number of clusters to be used to group the homes together
    """
    homes_subgraph = tsp_routines.complete_shortest_path_subgraph(graph,homes)
    home_clusters = k_cluster(homes_subgraph,num_clusters)
    # The source vertex is added to each of the clusters before determining the best dropoff location.
    # This is done so that vertices that are closer to the source are given higher preference as dropoff points.
    add_vertex_to_clusters(home_clusters,source)
    dropoff_vertices = get_dropoff_vertices(graph, home_clusters)
    # Add the source to the dropoff vertices
    dropoff_vertices.append(source)
    # Get rid of any repeating entries in the dropoff vertices
    dropoff_vertices = list(set(dropoff_vertices))
    # Construct the fully connected sub-graph with the dropoff vertices 
    # on which TSP is computed
    dropoff_subgraph = tsp_routines.complete_shortest_path_subgraph(graph,dropoff_vertices)
    tsp_route = tsp_routines.metric_mst_tsp(dropoff_subgraph,source)
    final_path = tsp_solution_to_path(graph,tsp_route)
    return final_path
    
def graph_from_input(filename):
    '''graph_from_input(str) --> nx.graph G, int source, np.ndarray[int] homes, dict locToIndex 
    Returns a graph created by reading the input file, with integer vertex labels
    Returns list of the home indices
    Returns a map from integer to the name associated with that node'''
    with open(filename, 'r') as f:
        G = nx.Graph()
        
        locToIndex = {} # maps location name to its index number
        homes = []
        lines = f.readlines()
        
        numLocations = int(lines[0])
        numTAs = int(lines[1])
        locations = lines[2].split()
    
        i = 0
        assert len(locations) == numLocations, "Number of locations must match specified value"
        for loc in locations:
            G.add_node(i)
            locToIndex[loc] = i
            i += 1
            
        TAhomes = lines[3].split()
        assert len(TAhomes) == numTAs, "Number of TA homes must match specified value"
        for home in TAhomes:
            homes.append(locToIndex[home])
        
        source = locToIndex[lines[4].strip()]
        
        row = 0
        for line in lines[5:]:
            line = line.split()
            for col in range(len(line)):
            
                if line[col] != 'x':  
                    G.add_edge(row, col)
                    weight = float(line[col])
                    G[row][col]['weight'] = weight
            row += 1
        
        indexToLoc = {v: k for k, v in locToIndex.items()}
        return G, source, homes, indexToLoc
    
def write_output_file(path, dropOffs, indexToName, filename):
    '''path is a list of integers that we follow in the car 
    dropOffs is a dictionary (int -> [home, home, ...] ) mapping nodes to the homes of the TAs that get off at that node 
    indexToName is a list of names corresponding to each index
    filename is a string filename that we write to  
    '''
    with open(filename, 'w') as f:
        for step in path:
            f.write(indexToName[step] + " ")
        
        f.write("\n")
        count = 0
        for key in dropOffs:
            if dropOffs[key] != []:
                count += 1
        f.write(str(count) + "\n")
        
        for key in dropOffs:
            if dropOffs[key] != []:
                f.write(indexToName[key] + " ")
                for elt in dropOffs[key]:
                    f.write(indexToName[elt] + " ");
                f.write("\n")
                
def eval_cost(G, path, dropoffs):
    '''
    eval_cost(nx.graph, np.ndarray, dict) -> int
    path is a iterable of integers that we follow in the car 
    dropOffs is a dictionary (int -> [home, home, ...] )
    '''
    assert isinstance(G, nx.Graph) , "G must be a graph"
    assert hasattr(path, '__iter__') , "path must be an array of integers"
    assert isinstance(dropoffs, dict) , "dropoffs must be a dictionary mapping node to homes"
    
    cost = 0
    prevNode = path[0]
    for node in path[1:]:
        cost += 2/3 * G[prevNode][node]['weight'] # weight of the edge from previous to next node
        prevNode = node
        
        for home in dropoffs[node]:
            cost += nx.astar_path_length(G, node, home)
                
    return cost

def nearest_dropoff(G,route,homes):
    """
    Returns a list representing the nearest dropoff point for each home
    G - graph with weights specified as distances
    route - array of indices representing vertices on path of Car
    homes - array of indices representing vertices which are TA homes
    """
    shortest_path_lens = dict(nx.all_pairs_dijkstra_path_length(G))
    
    wheredict = dict()
    for node in route:
        wheredict.update({node:[]}) #initialize a dropoff at each node in route. there could be places where noone is dropped.
    
    for h in homes:
        ls = shortest_path_lens[h]#shortest path from h to all nodes (dict)
        ls = {k:ls[k] for k in route}
        wheredict[min(ls, key=ls.get)].append(h)
    
    return wheredict

def solver(G, homes, source, k):
    """returns cost, dropoffs, route for a given k"""
    route = solve_by_clustering(G, homes, source, k)
    dropoffs = nearest_dropoff(G, route, homes)
    cost = eval_cost(G, route, dropoffs)
    return cost, dropoffs, route

def itersolver(G, homes, source, verbosity = 1):
    """starts with nclusters as len(homes), divides by 2 and iteratively solves halving each time"""
    iterations = int(np.log(len(homes))/np.log(2))
    i = 0
    ks = np.array([int(k) for k in np.linspace(1, len(homes), len(homes))])
    workerk = ks[-1]
    nextk = int(np.median(ks))
    cost, dropoffs, route = solver(G, homes, source, workerk)
    outcost = cost
    outdropoffs, outroute = dropoffs, route
    while i<iterations:
        ncost, ndropoffs, nroute = solver(G, homes, source, nextk)
        if outcost<ncost:
            ks = ks[ks>nextk]
            nextk = int(np.median(ks))
            if verbosity > 2:
                print('top, next - ', nextk)
        else:
            ks = ks[ks<=nextk]
            outcost, outdropoffs, outroute = ncost, ndropoffs, nroute
            nextk = int(np.median(ks))
            if verbosity > 2:
                print('bottom, next - ', nextk)
        i+=1
    return outcost, outdropoffs, outroute

In [42]:
def run_itersolver(fnames, verbosity = 1, redo = False):
    """run the solver on your set. save to file. return dict of fname and cost
    
    fnames - list of names to run on
    if redo then it will recalculate ones which have already been calculated
    """
    outdict = dict()
    n = len(fnames)
    for lmn, fname in enumerate(fnames):
        if not redo:
            if fname[:-3]+'.out' in set(os.listdir('./outputs')):
                print(fname[:-3], 'already processed. skipping')
                continue
        if verbosity >1:
            sys.stdout.write("\rProcessing input %i of "% lmn + str(n))
            sys.stdout.flush()
        G, source, homes, node_to_name_map = graph_from_input('./inputs/'+fname)
        cost, dropoffs, route = itersolver(G, homes, source, verbosity = verbosity)
        write_output_file(route, dropoffs, node_to_name_map, './outputs/'+fname[:-3]+'.out')
        outdict.update({fname:cost})
    return outdict


In [30]:
#with open('./inputlist.pkl', 'wb') as f:
#    pickle.dump(inputs, f)

In [43]:
with open('./inputlist.pkl', 'rb') as f:
    inputs = pickle.load(f)
fnames_eric = inputs[:316]
fnames_saagar = inputs[316:632]
fnames_arjun = inputs[632:]

In [ ]:
costdict = run_itersolver(fnames_eric,redo = False,verbosity = 2)